### 0. 참고논문
* 머신러닝 기반 음식점 추천시스템 설계 및 구현 - 최환석, 팽전, 이 우 섭

### 1. Khaiii 설치
* !git clone https://github.com/kakao/khaiii.git
* !mkdir khaiii/build
* !cd khaiii/build; cmake ..
* !cd khaiii/build; make all
* !cd khaiii/build; make resource
* !cd khaiii/build; make package_python
* !cd khaiii/build/package_python; pip install .

### 2. 나눔폰트 설치
* !apt install fonts-nanum

#### [참고]
* 설치폴더: /usr/share/fonts/truetype/nanum
* download: !wget http://cdn.naver.com/naver/NanumFont/fontfiles/NanumFont_TTF_ALL.zip

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fm._rebuild()
# colab에서는 런타임 다시시작 필요
# ubuntu에서는 reboot

### 3. data 보기

In [1]:
import numpy as np
import pandas as pd

In [4]:
xl = pd.ExcelFile('데이터매쓰 영화 데이터.xlsx')

In [5]:
xl.sheet_names

['KOBIS_역대_박스오피스(통합전산망_집계_기준)_202']

In [6]:
df = pd.read_excel('데이터매쓰 영화 데이터.xlsx',skiprows=4)

In [8]:
df.head(2)

,순위,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,국적,배급사,...,41,42,43,44,45,46,47,48,49,50
0,1,명량,2014-07-30,135757418810,17615686,1587,188720,한국,한국,(주)씨제이이엔엠,...,명량을 봤습니다. 유튜브 돌리다가 우연히 불멸의 이순신 클립 영상을 봤는데 cg가 ...,며칠동안 많은 고민 끝을 했다. 그 동안 꽤 많은 영화를 보았는데 과연 '어떤 영화...,모두들 알고 있듯이 1597년 임진왜란이 일어난지 6년\n\n매우 오래 지속된 전쟁...,"2014년 대종상 시상식에서 최우수작품상, 남우 주연상, 기획상, 기술상을,\n\n...","영화 명량, 참 오랜만에 다시보는 영화입니다. 6년전 이 영화가 개봉될 때 1700...","‘필사즉생 필생즉사’. 죽고자 하면 살 것이고, 살고자 하면 죽을 것이라는 뜻의 이...",오늘은 예전에 보았던 영화 중에 정말 재미있게 보았고 의미 있는 영화라고 \n\n생...,"국민의 존경을 한 몸에 받는 이순신,\n그는 누구를 위해서 싸웠는지 아직도 감이 잡...",명량해전의 승리는 영화에서 나오는 것처럼\n몇몇 백성과 장수들의 노력이나 수군덕만은...,당시 1000만명이나 넘긴 상당해서 리뷰 지금부터 관객수도 명량 바로 영화는 명량인...
1,2,극한직업,2019-01-23,139655543516,16266338,2003,292816,한국,한국,(주)씨제이이엔엠,...,"설연휴를 기념하여 오랜만에 극장을 찾았다. 맨날 보려고 했다가 취소하고, 보려고 했...",영화 극한직업은 매우 훌륭한 레시피로 만든 웰메이드 요리에 비유할 수 있겠다. 영화...,이 영화를 2번봤는데요. 스토리는 극한직업은 형사들이 치..(서론 생략)\n\n\n...,영화 극한직업 평점주기\n\n재미 ★★★★★\n\n연기력 ★★★★★\n\n내용구성 ...,우울한 겨울 끝에 여기저기서 들려오는 소리. \n\n \n\n“극한직업 재미있어요....,영화관에서 최근에 한국영화를 본 게 판도라였데 오랜만에 한국영화를 봤습니다. 마약범...,이번에 보고 온 영화는 2019년 1월 23일에 개봉한 한국 코미디 영화 <극한직업...,영화를 보는 시간은 크게 두가지가 있다고 생각한다. 영화라는 창작물 혹은 예술작품을...,"정말 아무 생각 없이 재미있다는 압도적인 평가 때문에, 설 연휴에 영화라도 한 편 ...",새해를 여는 웃음 바이러스! 마침 설연휴와 겹쳐 설명절 증후군을 겪는 사람들에게도 ...


In [9]:
df.shape

(100, 95)

### 4. review 받기

In [34]:
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [35]:
def get_review_content_url(title,year):
    baseurl = 'https://movie.naver.com/movie/search/result.naver?query=%s&section=movie&ie=utf8'
    pagenum = 1
    firsturl = baseurl % (urllib.parse.quote(title),)
    reviewurl = None
    while True:
        if pagenum == 1:
            url = firsturl
        else:
            url = '%s&page=%d' % (firsturl,pagenum)
#         print(url)
        data = urlopen(url).read()
        html = BeautifulSoup(data, 'html5lib')
        ul = html.find_all(attrs={'class': 'search_list_1'})
        if len(ul)>0:
            lilist = ul[0].find_all('li')
            for li in lilist:
                dt = li.find_all('dt')
                a1 = dt[0].find('a')
                t = a1.text
                dd = li.find_all(attrs={'class': 'etc'})
                a = dd[0].find_all('a')
                y = 0
                if re.search(r'^\d+$',a[-1].text):
                    y = int(a[-1].text)
                mlen = min(len(t),len(title))
                if title[:mlen] == t[:mlen]:
                    if y == year:
                        reviewurl = 'https://movie.naver.com' + re.sub('basic','review',a1.attrs['href'])
                        break
            if reviewurl is not None:
                break
        else:
            break
        pagenav = html.find(attrs={'class': 'pagenavigation'})
        tdlist = pagenav.find_all('td')
        lasttd = tdlist[-1]
        strnext = lasttd.text
        if strnext != '다음':
            break
        pagenum = pagenum+1
    if reviewurl is None:
        return reviewurl,None
    else:
        return reviewurl,t

def get_code(reviewurl):
    code = re.search(r'(?<=code=)[0-9]+',reviewurl).group(0)
    return code

def get_nid_list(reviewurl,maxreviews=None):
    nidlist = []
    page = 1
    while True:
        if page == 1:
            url = reviewurl
        else:
            url = '%s&page=%d' % (reviewurl,page)
        data = urlopen(url).read()
        html = BeautifulSoup(data, 'html5lib')
        ul = html.find(attrs={'class': 'rvw_list_area'})
        if ul is None:
            return []
        lilist = ul.find_all('li')
        for li in lilist:
            a = li.find_all('a')
            num = a[0].attrs['onclick']
            nid = re.search(r'\d+',num).group(0)
            nidlist.append(nid)
            if maxreviews != None and len(nidlist) >= maxreviews:
                return nidlist
        paging = html.find(attrs={'class': 'paging'})
        a = paging.find_all('a')
        if a[-1].text != '다음':
            break
        page = page + 1
    
    return nidlist

def get_review_list(nidlist,code):
    reviewprefix = 'https://movie.naver.com/movie/bi/mi/reviewread.naver'
    review_list = ['%s?nid=%s&code=%s&order=#tab'%(reviewprefix,x,code) for x in nidlist]
    return review_list

def get_review_text(review_list):
    review_texts = []
    scores = []
    titles = []
    nicknames = []
    clicks = []
    recommands = []
    dates = []
    for url in review_list:
        data = urlopen(url).read()
        html = BeautifulSoup(data, 'html5lib')
        tx_area = html.find(attrs={'class': 'user_tx_area'})
        star_score = html.find(attrs={'class': 'star_score'}).find('em')
        h_lst_tx = html.find(attrs={'class': 'h_lst_tx'})
        review_texts.append(tx_area.text.strip())
        name = html.find(attrs={'class': 'board_title'}).find_all('li')[-1].find('a').find('em')
        user_tx_info = html.find(attrs={'class': 'user_tx_info'}).find_all('span')
        wrt_date = html.find(attrs={'class': 'wrt_date'})
        scores.append(star_score.text)
        titles.append(h_lst_tx.text)
        nicknames.append(name.text)
        clicks.append(user_tx_info[0].find('em').text)
        recommands.append(user_tx_info[1].find('em').text)
        dates.append(wrt_date.text)

    print(len(nicknames),)
    df = pd.DataFrame({'별명': nicknames, '리뷰평점': scores, '리뷰조회': clicks,'리뷰추천': recommands,
                       '리뷰날짜': dates, '리뷰제목': titles, '리뷰': review_texts})
    return df

In [41]:
rnum = []
for i,row in df.iterrows():
    # 영화 선택
    title = row['영화명']
    year = row['개봉일'].year
    n = 0
    for j in range(1,51):
        a = row[j]
        if type(a) != str:
            n += 1
    rnum.append(n)

In [64]:
check_next_year = True
reviewurllist = []
for i,row in df.iterrows():
    # 영화 선택
    title = str(row['영화명'])
    year = row['개봉일'].year
    years = [year]
    if check_next_year:
        years = [year,year+1,year-1]
    for y in years:
        reviewurl,title2 = get_review_content_url(title,y)
        if reviewurl is not None:
            break
    reviewurllist.append([title,str(year),reviewurl])

In [65]:
df.loc[df.영화명=='7번방의 선물']

,순위,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,국적,배급사,...,41,42,43,44,45,46,47,48,49,50
8,9,7번방의 선물,2013-01-23,91437368170,12812144,866,167013,한국,한국,(주)넥스트엔터테인먼트월드(NEW),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
[x for x in reviewurllist if x[2] is None]

[['미션임파서블:고스트프로토콜', '2011', None],
 ['스파이더맨: 홈 커밍', '2017', None],
 ['월드 워 Z', '2013', None],
 ['쿵푸팬더 2', '2011', None],
 ['조선명탐정 : 각시투구꽃의 비밀', '2011', None],
 ['군도: 민란의 시대', '2014', None]]

In [86]:
nidlist = []
for i,a in enumerate(reviewurllist):
    title,year,reviewurl = a
    code = get_code(reviewurl)
    nreviews = rnum[i]
    if nreviews ==0:
        continue
    nidlist = get_nid_list(reviewurl,maxreviews=nreviews)
    review_list = get_review_list(nidlist,code)
    df_review = get_review_text(review_list)
    print(i)
    k = 0
    for j in range(1,51):
        b = df.loc[i,j]
        if type(b) != str:
            df.loc[i,j] = df_review.loc[k,'리뷰']
            k += 1

20
3
20
4
50
5
50
6


KeyboardInterrupt: 

In [85]:
nidlist

[['3705815',
  '3708914',
  '3681106',
  '3691857',
  '3681103',
  '3710472',
  '3709927',
  '3714107',
  '3705770',
  '3713923',
  '3723203',
  '3703931',
  '3346691',
  '3709997',
  '3715316',
  '3723550',
  '3716405',
  '3704864',
  '3717000',
  '4302410']]

In [80]:
df.loc[3]

순위                                                        4
영화명                                                    국제시장
개봉일                                     2014-12-17 00:00:00
매출액                                            110944607730
관객수                                                14263980
스크린수                                                   1044
상영횟수                                                 212696
대표국적                                                     한국
국적                                                       한국
배급사                                               (주)씨제이이엔엠
장르                                                      드라마
관람객                                                    9.16
평론가                                                    5.81
네티즌                                                    9.02
남자                                                     9.22
여자                                                     9.11
10대                                     